In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

Python REPL can execute arbitrary code. Use with caution.


In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs - {today}"

In [4]:
session_id = '882432'

In [5]:
task = "Consider the first week of June 2019 - find 3 pitchers who's changeups were most similar to Justin Verlander's"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Plan'}}
----
Distance to neareast plan: 0.3152533769607544
Modifying nearest plan with User input
{'Plan': {'messages': [HumanMessage(content="\n1. Use the playerid_lookup function to get Justin Verlander's 'key_mlbam' player ID:\n\n```python\nfrom pybaseball import playerid_lookup\nverlander_id = playerid_lookup('verlander', 'justin')['key_mlbam'].iloc[0]\n```\n\n2. Use the statcast function to get pitch data for the first week of June 2019:\n\n```python\nfrom pybaseball import statcast\npitch_data = statcast('2019-06-01', '2019-06-07')\n```\n\n3. Create a new DataFrame with only changeup pitches and relevant features:\n\n```python\nchange_data = pitch_data.loc[pitch_data['pitch_type'] == 'CH', ['player_name', 'release_speed', 'release_spin', 'release_extension']]\n```\n\n4. Group the change_data by 'player_name' and calculate the mean of each feature to create an average changeup vector for each pitcher:\n\n```python\navg_changes = change_data.groupby('playe

In [6]:
task = "looks good"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Convert'}}
----
{'Convert': {'code': '\n# Import necessary libraries\nfrom pybaseball import playerid_lookup, statcast\nfrom sklearn.neighbors import NearestNeighbors\n\n# 1. Get Justin Verlander\'s \'key_mlbam\' player ID\nverlander_id = playerid_lookup(\'verlander\', \'justin\')[\'key_mlbam\'].iloc[0]\n\n# 2. Get pitch data for the first week of June 2019\npitch_data = statcast(\'2019-06-01\', \'2019-06-07\')\n\n# 3. Create a new DataFrame with only changeup pitches and relevant features\nchange_data = pitch_data.loc[pitch_data[\'pitch_type\'] == \'CH\', [\'player_name\', \'release_speed\', \'release_spin\', \'release_extension\']]\n\n# 4. Group the change_data by \'player_name\' and calculate the mean of each feature\navg_changes = change_data.groupby(\'player_name\').mean().reset_index()\n\n# 5. Split the avg_changes data into X (features) and y (player names)\nX = avg_changes[[\'release_speed\', \'release_spin\', \'release_extension\']]\ny = avg_changes[\

  0%|          | 0/7 [00:00<?, ?it/s]

Result: Gathering player lookup table. This may take a moment.

Executing: # 2. Get pitch data for the first week of June 2019
pitch_data = statcast('2019-06-01', '2019-06-07')


100%|██████████| 7/7 [00:18<00:00,  2.62s/it]


Result: This is a large query, it may take a moment to complete

Executing: # 3. Create a new DataFrame with only changeup pitches and relevant features
change_data = pitch_data.loc[pitch_data['pitch_type'] == 'CH', ['player_name', 'release_speed', 'release_spin', 'release_extension']]
Result: KeyError("['release_spin'] not in index")
Executing: 
# 3. Create a new DataFrame with only changeup pitches and relevant features
change_data = pitch_data.loc[pitch_data['pitch_type'] == 'CH', ['player_name', 'release_speed', 'release_extension']]

Executing: # 4. Group the change_data by 'player_name' and calculate the mean of each feature
avg_changes = change_data.groupby('player_name').mean().reset_index()
Executing: # 5. Split the avg_changes data into X (features) and y (player names)
X = avg_changes[['release_speed', 'release_spin', 'release_extension']]
y = avg_changes['player_name']
Result: KeyError("['release_spin'] not in index")
Executing: 
# 5. Split the avg_changes data into X (feat

In [7]:
task = "yes"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Memorize'}}
----
{'Memorize': {'messages': [AIMessage(content='Thank you, task has been commited to memory')], 'previous_node': 'Memorize'}}
----
Thank you, task has been commited to memory
